In [1]:
df = spark.read.parquet("../data/silver_np/orders")

df.filter("order_ts >= '2026-01-10' AND order_ts < '2026-01-11'") \
  .groupBy("country") \
  .count() \
  .explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (7)
+- HashAggregate (6)
   +- Exchange (5)
      +- HashAggregate (4)
         +- Project (3)
            +- Filter (2)
               +- Scan parquet  (1)


(1) Scan parquet 
Output [2]: [order_ts#3, country#5]
Batched: true
Location: InMemoryFileIndex [file:/Users/nptan2005/SourceCode/Python/spark401_colab/data/silver_np/orders]
PushedFilters: [IsNotNull(order_ts), GreaterThanOrEqual(order_ts,2026-01-10 00:00:00.0), LessThan(order_ts,2026-01-11 00:00:00.0)]
ReadSchema: struct<order_ts:timestamp,country:string>

(2) Filter
Input [2]: [order_ts#3, country#5]
Condition : ((isnotnull(order_ts#3) AND (order_ts#3 >= 2026-01-10 00:00:00)) AND (order_ts#3 < 2026-01-11 00:00:00))

(3) Project
Output [1]: [country#5]
Input [2]: [order_ts#3, country#5]

(4) HashAggregate
Input [1]: [country#5]
Keys [1]: [country#5]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#25L]
Results [2]: [country#5, count#26L]

(5) Exchange
Input [2]: [country#5

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

spark = (SparkSession.builder
  .appName("q_lab2d_partitioned_write")
  .config("spark.sql.shuffle.partitions", "50")   # giảm mặc định 200 cho máy local
  .getOrCreate())

spark.read.parquet("../data/silver_p_lab2d/orders") \
  .where(col("dt") == "2026-01-10") \
  .groupBy("country").count().explain("formatted")

spark.stop()

26/01/12 15:43:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


== Physical Plan ==
AdaptiveSparkPlan (6)
+- HashAggregate (5)
   +- Exchange (4)
      +- HashAggregate (3)
         +- Project (2)
            +- Scan parquet  (1)


(1) Scan parquet 
Output [2]: [country#73, dt#79]
Batched: true
Location: InMemoryFileIndex [file:/Users/nptan2005/SourceCode/Python/spark401_colab/data/silver_p_lab2d/orders]
PartitionFilters: [isnotnull(dt#79), (dt#79 = 2026-01-10)]
ReadSchema: struct<country:string>

(2) Project
Output [1]: [country#73]
Input [2]: [country#73, dt#79]

(3) HashAggregate
Input [1]: [country#73]
Keys [1]: [country#73]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#95L]
Results [2]: [country#73, count#96L]

(4) Exchange
Input [2]: [country#73, count#96L]
Arguments: hashpartitioning(country#73, 50), ENSURE_REQUIREMENTS, [plan_id=45]

(5) HashAggregate
Input [2]: [country#73, count#96L]
Keys [1]: [country#73]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#94L]
Results [2]: [country#73, count(1)#94L AS coun

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date

spark = (SparkSession.builder
  .appName("q2_lab2d_partitioned_write")
  .config("spark.sql.shuffle.partitions", "50")   # giảm mặc định 200 cho máy local
  .getOrCreate())

spark.read.parquet("../data/silver_p_lab2d/orders") \
  .where(to_date(col("order_ts")) == "2026-01-10") \
  .groupBy("country").count().explain("formatted")

spark.stop()

== Physical Plan ==
AdaptiveSparkPlan (7)
+- HashAggregate (6)
   +- Exchange (5)
      +- HashAggregate (4)
         +- Project (3)
            +- Filter (2)
               +- Scan parquet  (1)


(1) Scan parquet 
Output [3]: [order_ts#100, country#102, dt#108]
Batched: true
Location: InMemoryFileIndex [file:/Users/nptan2005/SourceCode/Python/spark401_colab/data/silver_p_lab2d/orders]
PushedFilters: [IsNotNull(order_ts), GreaterThanOrEqual(order_ts,2026-01-10 00:00:00.0), LessThan(order_ts,2026-01-11 00:00:00.0)]
ReadSchema: struct<order_ts:timestamp,country:string>

(2) Filter
Input [3]: [order_ts#100, country#102, dt#108]
Condition : ((isnotnull(order_ts#100) AND (order_ts#100 >= 2026-01-10 00:00:00)) AND (order_ts#100 < 2026-01-11 00:00:00))

(3) Project
Output [1]: [country#102]
Input [3]: [order_ts#100, country#102, dt#108]

(4) HashAggregate
Input [1]: [country#102]
Keys [1]: [country#102]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#124L]
Results [2]: [cou

In [10]:
## 1) Write partitioned nhưng không repartition
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

spark = (SparkSession.builder
  .appName("lab2e_write_no_repartition")
  .config("spark.sql.shuffle.partitions", "50")
  .getOrCreate())

df = (spark.read.parquet("../data/silver/orders_enriched")
      .withColumn("dt", to_date(col("order_ts"))))

(df.write.mode("overwrite")
   .partitionBy("dt")
   .parquet("../data/silver_p_lab2e/no_repart"))

spark.stop()

26/01/12 16:26:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/12 16:26:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/12 16:26:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
26/01/12 16:26:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
26/01/12 16:26:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
26/01/12 16:26:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
26/01/12 16:26:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row grou

In [11]:
## 2) Write partitioned và repartition theo dt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

spark = (SparkSession.builder
  .appName("lab2e_write_repartition_dt")
  .config("spark.sql.shuffle.partitions", "50")
  .getOrCreate())

df = (spark.read.parquet("../data/silver/orders_enriched")
      .withColumn("dt", to_date(col("order_ts")))
      .repartition("dt"))   # key điểm

(df.write.mode("overwrite")
   .partitionBy("dt")
   .parquet("../data/silver_p_lab2e/repart_dt"))

spark.stop()

26/01/12 16:29:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/12 16:29:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/12 16:29:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
26/01/12 16:29:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
26/01/12 16:29:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
26/01/12 16:29:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
26/01/12 16:29:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row grou

In [13]:
## C) Tại sao trong Query 1 bạn vẫn thấy Exchange dù đã prune?

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

spark = (SparkSession.builder
  .appName("lab2e_write_explain")
  .config("spark.sql.shuffle.partitions", "50")
  .getOrCreate())

spark.read.parquet("../data/silver_p_lab2d/orders") \
  .where(col("dt") == "2026-01-10") \
  .groupBy("dt").count().explain("formatted")

spark.stop()

26/01/12 16:35:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


== Physical Plan ==
AdaptiveSparkPlan (5)
+- HashAggregate (4)
   +- Exchange (3)
      +- HashAggregate (2)
         +- Scan parquet  (1)


(1) Scan parquet 
Output [1]: [dt#163]
Batched: true
Location: InMemoryFileIndex [file:/Users/nptan2005/SourceCode/Python/spark401_colab/data/silver_p_lab2d/orders]
PartitionFilters: [isnotnull(dt#163), (dt#163 = 2026-01-10)]
ReadSchema: struct<>

(2) HashAggregate
Input [1]: [dt#163]
Keys [1]: [dt#163]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#179L]
Results [2]: [dt#163, count#180L]

(3) Exchange
Input [2]: [dt#163, count#180L]
Arguments: hashpartitioning(dt#163, 50), ENSURE_REQUIREMENTS, [plan_id=172]

(4) HashAggregate
Input [2]: [dt#163, count#180L]
Keys [1]: [dt#163]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#178L]
Results [2]: [dt#163, count(1)#178L AS count#165L]

(5) AdaptiveSparkPlan
Output [2]: [dt#163, count#165L]
Arguments: isFinalPlan=false




In [14]:
# 1 ví dụ nữa bạn muốn: kết hợp partition column + xử lý partition trên column (chuẩn “data lake”)

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

spark = (SparkSession.builder
  .appName("lab2f_partition_and_file_control")
  .config("spark.sql.shuffle.partitions", "50")
  .getOrCreate())

src = spark.read.parquet("../data/silver/orders_enriched") \
  .withColumn("dt", to_date(col("order_ts")))

# 1) Ghi partitioned, kiểm soát số file:
# - repartition theo dt để gom theo ngày
# - thêm coalesce trong từng dt là khó (Spark không coalesce per-partitionBy),
#   nên thường làm: repartition(dt, N) để mỗi dt có ~N files
out = "../data/silver_p_lab2f/orders"

(src
  .repartition(30, "dt")   # 30 partitions execution cho toàn dataset (thường ~= số ngày)
  .write.mode("overwrite")
  .partitionBy("dt")
  .parquet(out)
)

# 2) Đọc lại: pruning
q = (spark.read.parquet(out)
     .where(col("dt") == "2026-01-10")
     .groupBy("country").count())

q.explain("formatted")
q.show()

spark.stop()

26/01/12 16:43:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/12 16:43:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/12 16:43:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
26/01/12 16:43:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
26/01/12 16:43:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
26/01/12 16:43:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
26/01/12 16:43:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row grou

== Physical Plan ==
AdaptiveSparkPlan (6)
+- HashAggregate (5)
   +- Exchange (4)
      +- HashAggregate (3)
         +- Project (2)
            +- Scan parquet  (1)


(1) Scan parquet 
Output [2]: [country#199, dt#205]
Batched: true
Location: InMemoryFileIndex [file:/Users/nptan2005/SourceCode/Python/spark401_colab/data/silver_p_lab2f/orders]
PartitionFilters: [isnotnull(dt#205), (dt#205 = 2026-01-10)]
ReadSchema: struct<country:string>

(2) Project
Output [1]: [country#199]
Input [2]: [country#199, dt#205]

(3) HashAggregate
Input [1]: [country#199]
Keys [1]: [country#199]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#221L]
Results [2]: [country#199, count#222L]

(4) Exchange
Input [2]: [country#199, count#222L]
Arguments: hashpartitioning(country#199, 50), ENSURE_REQUIREMENTS, [plan_id=250]

(5) HashAggregate
Input [2]: [country#199, count#222L]
Keys [1]: [country#199]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#220L]
Results [2]: [country#199,